In [29]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, ecdab3fa-156d-4e13-b964-8c4089366615, 31, Finished, Available, Finished)

In [30]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news LIMIT 1000")
display(df)

StatementMeta(, ecdab3fa-156d-4e13-b964-8c4089366615, 32, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 46e400e8-6b31-4428-b8f8-6fc9e449763b)

In [31]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, ecdab3fa-156d-4e13-b964-8c4089366615, 33, Finished, Available, Finished)

In [32]:
#import the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, ecdab3fa-156d-4e13-b964-8c4089366615, 34, Finished, Available, Finished)

In [33]:
#apply the model to our dataframe
result = model.transform(df)

StatementMeta(, ecdab3fa-156d-4e13-b964-8c4089366615, 35, Finished, Available, Finished)

In [34]:
display(result)

StatementMeta(, ecdab3fa-156d-4e13-b964-8c4089366615, 36, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d28ec3bb-b32d-46bc-b1fd-4de2a5aa47a7)

In [35]:
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, ecdab3fa-156d-4e13-b964-8c4089366615, 37, Finished, Available, Finished)

In [36]:
sentiment_df_final = sentiment_df.drop("response","error")

StatementMeta(, ecdab3fa-156d-4e13-b964-8c4089366615, 38, Finished, Available, Finished)

In [37]:
display(sentiment_df_final)

StatementMeta(, ecdab3fa-156d-4e13-b964-8c4089366615, 39, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9f684240-a342-476e-94e8-98435a8f7768)

In [38]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'
    
    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:
    print("Table Already exists")

    # Create or replace a temporary view
    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""
        MERGE INTO {table_name} AS target_table
        USING vw_sentiment_df_final AS source_view
        ON source_view.url = target_table.url
        WHEN MATCHED AND (
            source_view.title <> target_table.title OR
            source_view.description <> target_table.description OR
            source_view.category <> target_table.category OR
            source_view.image <> target_table.image OR
            source_view.provider <> target_table.provider OR
            source_view.datePublished <> target_table.datePublished
        )
        THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)

StatementMeta(, ecdab3fa-156d-4e13-b964-8c4089366615, 40, Finished, Available, Finished)